In [1]:
using BenchmarkTools

In [313]:
gensym(:scrabble)

Symbol("##scrabble#346")

In [413]:
module compile

macro feature(ex, description::String="")
    Expr(:call, :Feature, Expr(:quote, ex), Expr(:quote, Expr(:quote, description)))
end

macro feature(ex, description::Expr)
    Expr(:call, :Feature, Expr(:quote, ex), Expr(:quote, description))
end

cacheify!(setup, expr, input_var::Symbol) = expr

function cacheify!(setup::Vector{Expr}, expr::Expr, input_var::Symbol)
    if expr.head == :call && length(expr.args) == 2 && expr.args[2] == input_var
        cached_varname = gensym(expr.args[1])
        push!(setup, :($cached_varname = $(copy(expr))))
        cached_varname
    else
        for (i, child) in enumerate(expr.args)
            expr.args[i] = cacheify!(setup, child, input_var)
        end
        expr
    end
end

function cacheify(expr::Expr, var::Symbol)
    setup = Expr[]
    expr = cacheify!(setup, expr, var)
    Expr(:block, setup..., expr)
end

immutable Feature
    expr::Expr
    args::Vector{Tuple{Symbol, Expr}}
    length::Int
    description::Expr
    function Feature(expr::Expr, description::Expr)
        rhs = expr.args[1]
        args = Tuple{Symbol, Expr}[]
        for ex in expr.args[2:end]
            @assert ex.head == :(=)
            push!(args, (ex.args[1], ex.args[2]))
        end
        len = prod(length(eval(ex[2])) for ex in args)
        new(rhs, args, len, description)
    end
end
       
    immutable FeatureSet{F}
        features::Vector{Feature}
        evaluate::F
        descriptions::Vector{String}
    end
    
    function loop_over(args::Vector{Tuple{Symbol, Expr}}, inner::Expr)
        for (var, expr) in reverse(args)
            inner = quote
                for $var in $expr
                    $inner
                end
            end
        end
        inner
    end
 
    
    function compile_test(features::Vector{Feature})
        num_features = sum([f.length for f in features])
                values = gensym(:values)
        loop_var = gensym(:i)
        result = quote
            $values = falses($(num_features))
            $loop_var = 1
        end
        for (i, feat) in enumerate(features)
            inner = quote
                $values[$loop_var] = $(feat.expr)
                $loop_var += 1
            end
            push!(result.args, loop_over(feat.args, inner))
        end
                push!(result.args, values)
        result = cacheify(result, :word)
        println(result)
        result
    end
    
    function compile_descriptions(features::Vector{Feature})
        num_features = sum([f.length for f in features])
        values = gensym(:values)
        loop_var = gensym(:i)
        result = quote
            $values = Vector{String}($(num_features))
            $loop_var = 1
        end
        for (i, feat) in enumerate(features)
            inner = quote
                $values[$loop_var] = $(feat.description)
                $loop_var += 1
            end
            push!(result.args, loop_over(feat.args, inner))
        end
        push!(result.args, values)
        result
    end

    function FeatureSet(features)
        test_expr = compile_test(features)
        description_expr = compile_descriptions(features)
        FeatureSet(features,
            eval(:((word) -> $(test_expr))),
            eval(:(() -> $(description_expr)))())
    end

    function scrabble_score(word::String)
        sum(scrabble_score(c) for c in word)
    end

    scrabble_score(char::Char) = SCRABBLE_SCORES[char]

    const SCRABBLE_SCORES = Dict{Char, Int}(
        'e' => 1,
        'a' => 1,
        'i' => 1,
        'o' => 1,
        'n' => 1,
        'r' => 1,
        't' => 1,
        'l' => 1,
        's' => 1,
        'u' => 1,
        'd' => 2,
        'g' => 2,
        'b' => 3,
        'c' => 3,
        'm' => 3,
        'p' => 3,
        'f' => 4,
        'h' => 4,
        'v' => 4,
        'w' => 4,
        'y' => 4,
        'k' => 5,
        'j' => 8,
        'x' => 8,
        'q' => 10,
        'z' => 10
    )

function allfeatures()
    features = Feature[]
    push!(features, @feature((scrabble_score(word) == j for j in 1:26), "scrabble score $j"))
    push!(features, @feature((c in word for c in 'a':'z'), "contains $c"))
    push!(features, @feature((length(word) >= j && word[j] == c for c in 'a':'z', j in 1:26), "contains $c at index $j"))
    features
end
    

end
    



compile

In [414]:
compile.allfeatures()

3-element Array{compile.Feature,1}:
 compile.Feature(:(scrabble_score(word) == j),Tuple{Symbol,Expr}[(:j,:(1:26))],26,:("scrabble score $(j)"))                                 
 compile.Feature(:(c in word),Tuple{Symbol,Expr}[(:c,:('a':'z'))],26,:("contains $(c)"))                                                    
 compile.Feature(:(length(word) >= j && word[j] == c),Tuple{Symbol,Expr}[(:c,:('a':'z')),(:j,:(1:26))],676,:("contains $(c) at index $(j)"))

In [415]:
fset = compile.FeatureSet(compile.allfeatures())

begin 
    ##scrabble_score#404 = scrabble_score(word)
    ##length#405 = length(word)
    begin  # In[413], line 93:
        ##values#402 = falses(728) # In[413], line 94:
        ##i#403 = 1
        begin  # In[413], line 79:
            for j = 1:26 # In[413], line 80:
                begin  # In[413], line 98:
                    ##values#402[##i#403] = ##scrabble_score#404 == j # In[413], line 99:
                    ##i#403 += 1
                end
            end
        end
        begin  # In[413], line 79:
            for c = 'a':'z' # In[413], line 80:
                begin  # In[413], line 98:
                    ##values#402[##i#403] = c in word # In[413], line 99:
                    ##i#403 += 1
                end
            end
        end
        begin  # In[413], line 79:
            for c = 'a':'z' # In[413], line 80:
                begin  # In[413], line 79:
                    for j = 1:26 # In[413], line 80:
                        begin  # In[413], line 98:
  

compile.FeatureSet{compile.##7#8}(compile.Feature[compile.Feature(:(##scrabble_score#404 == j),Tuple{Symbol,Expr}[(:j,:(1:26))],26,:("scrabble score $(j)")),compile.Feature(:(c in word),Tuple{Symbol,Expr}[(:c,:('a':'z'))],26,:("contains $(c)")),compile.Feature(:(##length#405 >= j && word[j] == c),Tuple{Symbol,Expr}[(:c,:('a':'z')),(:j,:(1:26))],676,:("contains $(c) at index $(j)"))],compile.#7,String["scrabble score 1","scrabble score 2","scrabble score 3","scrabble score 4","scrabble score 5","scrabble score 6","scrabble score 7","scrabble score 8","scrabble score 9","scrabble score 10"  …  "contains z at index 17","contains z at index 18","contains z at index 19","contains z at index 20","contains z at index 21","contains z at index 22","contains z at index 23","contains z at index 24","contains z at index 25","contains z at index 26"])

In [416]:
vals = fset.evaluate("hello")

728-element BitArray{1}:
 false
 false
 false
 false
 false
 false
 false
  true
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [417]:
[fset.descriptions[i] for i in 1:length(fset.descriptions) if vals[i]]

10-element Array{String,1}:
 "scrabble score 8"     
 "contains e"           
 "contains h"           
 "contains l"           
 "contains o"           
 "contains e at index 2"
 "contains h at index 1"
 "contains l at index 3"
 "contains l at index 4"
 "contains o at index 5"

In [412]:
word = "hello"
@benchmark $fset.evaluate($word)

BenchmarkTools.Trial: 
  memory estimate:  240.00 bytes
  allocs estimate:  4
  --------------
  minimum time:     7.851 μs (0.00% GC)
  median time:      8.123 μs (0.00% GC)
  mean time:        8.307 μs (0.00% GC)
  maximum time:     26.918 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     4
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [306]:
function manual(word)
    values = BitArray{1}()
    append!(values, BitArray{1}(c in word for c in 'a':'z'))
    append!(values, (length(word) >= j && word[j] == c for c in 'a':'z' for j in 1:26))
    values
#     i = 1
#     for v in (c in word for c in 'a':'z')
#         values[i] = v
#         i += 1
#     end
#     for v in (length(word) >= j && word[j] == c for c in 'a':'z' for j in 1:26)
#         values[i] = v
#         i += 1
#         end
#     values
end
    

manual (generic function with 1 method)

In [307]:
word = "hello"
@benchmark manual($word)

LoadError: LoadError: MethodError: Cannot `convert` an object of type Base.Generator{StepRange{Char,Int64},##52#55{String}} to an object of type BitArray{1}
This may have arisen from a call to the constructor BitArray{1}(...),
since type constructors fall back to convert methods.
while loading In[307], in expression starting on line 182